<a href="https://colab.research.google.com/github/zeqkik/LLM-Fine-Tuning/blob/main/fine_tuning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitsandbytes
!pip install trl

In [4]:
import pandas as pd
import numpy as np
import torch
import random
import time
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model
import google.generativeai as genai
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import drive
from datasets import Dataset

In [5]:
import bitsandbytes
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model

In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Base Model(Gemma 7B-IT) Evaluation

In [7]:
hf_token = userdata.get('HUGGIN_FACE_TOKEN')
login(hf_token)

In [ ]:
model_id = "google/gemma-7b-it"

tokenizer_gemma = AutoTokenizer.from_pretrained(model_id)

model_gemma = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True
)


In [34]:
def generate_response(model, prompt_text):
  input_ids = tokenizer_gemma(prompt_text, return_tensors="pt").to(model.device)

  outputs = model.generate(
        **input_ids,
        max_new_tokens=40,
    )

  response = tokenizer_gemma.decode(outputs[0], skip_special_tokens=True)
  cleaned_response = response
  if cleaned_response.startswith(prompt_text):
      cleaned_response = cleaned_response[len(prompt_text):].strip()

  return cleaned_response

In [15]:
prompt_1 = """
Você está jogando Uno.
A carta no descarte é: Vermelho 7
Minha mão tem as seguintes cartas: [Azul 3, Vermelho 8, Verde 5, +2 Amarelo].
Qual carta você pode jogar?
"""

In [16]:
prompt_2 = """
Você está jogando Uno.
A carta no descarte é: Vermelho +2.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
"""

In [17]:
prompt_3 = """
Você está jogando Uno.
A carta no descarte é: Azul 4.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?
"""

In [18]:
prompt_4 = """
Você está jogando Uno.
Carta no descarte: Azul 8,
Cartas na mão: [Coringa,Vermelho 8,Amarelo +2,Verde 7,Vermelho 3,Vermelho 9,Amarelo 4].
Qual carta você pode jogar?
"""

In [19]:
prompt_5 = """
Você está jogando Uno.
Carta no descarte: Vermelho 2,
Cartas na mão: [Azul 3,Vermelho 4,Amarelo 4,Amarelo Inverter,Verde +2,Azul 6,Vermelho 3].
Qual carta você pode jogar?
"""

In [50]:
response_1 = generate_response(model_gemma, prompt_1 )
response_2 = generate_response(model_gemma, prompt_2 )
response_3 = generate_response(model_gemma, prompt_3 )
response_4 = generate_response(model_gemma, prompt_4 )
response_5 = generate_response(model_gemma, prompt_5 )

In [51]:
prompts = [prompt_1, prompt_2, prompt_3, prompt_4, prompt_5]
responses = [response_1, response_2, response_3, response_4, response_5]

In [52]:
# Model examples

for prompt, response in zip(prompts, responses):
  print(f"**Prompt:**\n{prompt}\n")
  print(f"**Answer:**\n{response}\n")
  print("-" * 50)


**Prompt:**

Você está jogando Uno.
A carta no descarte é: Vermelho 7
Minha mão tem as seguintes cartas: [Azul 3, Vermelho 8, Verde 5, +2 Amarelo].
Qual carta você pode jogar?


**Answer:**
**Resposta:** A carta que você pode jogar é o Verde 5.

--------------------------------------------------
**Prompt:**

Você está jogando Uno.
A carta no descarte é: Vermelho +2.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?


**Answer:**
A. Verde 0
B. Azul 1
C. Vermelho +2

A resposta para a questão é a carta Verde 0.

A carta Vermelho +2 foi descar

--------------------------------------------------
**Prompt:**

Você está jogando Uno.
A carta no descarte é: Azul 4.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?


**Answer:**
**Resposta:** A carta que você pode jogar é a Azul 1.

--------------------------------------------------
**Prompt:**

Você está jogando Uno.
Carta no descarte: Azul 8,
Car

## Dataset Creation



In [ ]:
!pip install -q -U google-generativeai

In [ ]:
api_key = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=api_key)

In [ ]:
uno_rules = """
**Regras Oficiais do Jogo UNO:**
1.  **Objetivo:** Livrar-se de todas as cartas.
2.  **Jogada Válida:** Uma carta só pode ser jogada se combinar com a cor, o número ou o símbolo da carta que está no topo do monte de descarte.
3.  **Carta de Número (0-9):** Combina cor ou número.
4.  **Carta '+2':** Combina cor. O próximo jogador deve comprar duas cartas e perde a vez.
5.  **Carta 'Bloquear' (Skip):** Combina cor. O próximo jogador perde a vez.
6.  **Carta 'Inverter' (Reverse):** Combina cor. Inverte o sentido do jogo.
7.  **Carta 'Trocar Cor' (Wild Card):** Pode ser jogada a qualquer momento, independentemente da carta no descarte. O jogador que a joga escolhe a próxima cor (vermelho, azul, verde ou amarelo).
8.  **Carta '+4' (Wild Draw Four):** Pode ser jogada a qualquer momento, *mas apenas se o jogador não tiver nenhuma carta da cor atual no descarte* (exceto Coringas normais). O jogador que a joga escolhe a próxima cor. O próximo jogador deve comprar quatro cartas e perde a vez.
9.  **Comprar Cartas:** Se um jogador não tiver uma carta válida para jogar, ele deve comprar uma carta do baralho de compra. Se a carta comprada for jogável, ele pode jogá-la imediatamente. Caso contrário, a vez passa para o próximo jogador.
10. **Aviso 'UNO!':** Quando um jogador tem apenas uma carta na mão, ele deve dizer "UNO!". Se ele não disser e outro jogador o pegar antes da vez do próximo jogador, ele deve comprar duas cartas.
"""

In [ ]:
gemini_table_generation_prompt = f"""
Você é um especialista em jogos de cartas, especificamente em UNO. Sua tarefa é criar uma tabela contendo situações de jogo de UNO (entradas) e as jogadas/resultados corretos para cada situação (saídas), baseando-se estritamente nas regras oficiais do jogo que serão fornecidas.

**Objetivo:** Gerar um conjunto de dados para treinar uma IA a jogar UNO, cobrindo a maior variedade possível de situações e regras.

---

{uno_rules}

---

**Instruções Detalhadas para a Geração da Tabela:**

1.  **Formato da Tabela:** Gere uma tabela com **10 exemplos**. Cada exemplo deve ter 2 colunas:
    * **Cenário (prompt):** Uma descrição concisa da situação do jogo. Siga este formato *exato*: `Carta no descarte: [COR/TIPO], Cartas na mão: [[COR/TIPO], [COR/TIPO], ...]. Qual(is) carta(s) da mão você pode jogar ou o que deve fazer?`
    * **Resposta (completion):** A jogada ou o resultado correto para o cenário, **extremamente conciso**. Se houver uma breve justificativa de uma frase (ex: "Combina cor/número", "Causa compra de cartas"), inclua-a após a jogada/resultado. **Não inclua qualquer frase introdutória.**

2.  **Variedade de Cenários:** Garanta que os exemplos abordem uma AMPLA VARIEDADE de situações e testem **diferentes regras** listadas acima.

3.  **Formato da Tabela de Saída (Exemplo):**
    ```
    Cenário (prompt) | Resposta (completion)
    --- | ---
    Carta no descarte: Vermelho 7, Cartas na mão: [Azul 3, Vermelho 8, Verde 5, +2 Amarelo]. Qual(is) carta(s) da mão você pode jogar ou o que deve fazer? | **Vermelho 8**. Combina com a cor Vermelha.
    Carta no descarte: Azul +2, Cartas na mão: [Verde 0, Azul 1, Bloquear Vermelho, Coringa]. O que acontece com você nesta rodada? | Você deve **comprar duas cartas e perder a sua vez**.
    ```
    Gere a tabela completa agora com 250 exemplos, seguindo todas as instruções.
"""

In [ ]:
model_gemini_pro = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
colors = ['Vermelho', 'Azul', 'Verde', 'Amarelo']
numbers = [str(i) for i in range(10)]
action_cards = ['+2', 'Bloquear', 'Inverter']
wild_cards = ['Coringa', '+4']



In [ ]:
full_uno_deck = []
for color in colors:
    full_uno_deck.append(f"{color} 0") # One 0 for each color
    for num in numbers[1:]: # Two of each number[1 to 9] for each color
        full_uno_deck.append(f"{color} {num}")
        full_uno_deck.append(f"{color} {num}")

# Action Cards (Two of each color for +2, Block, Reverse)
for color in colors:
    for action in action_cards:
        full_uno_deck.append(f"{color} {action}")
        full_uno_deck.append(f"{color} {action}")

# Wild cards (Four of each color)
for _ in range(4):
  for wild_card in wild_cards:
    full_uno_deck.append(f"{wild_card}")


In [ ]:
def generate_random_hand_and_discard(deck):
  remaining_deck = list(deck)
  num_cards = random.randint(3,7)

  hand = random.sample(deck, num_cards)

  for card in hand:
    remaining_deck.remove(card)

  discard_card = random.choice(remaining_deck)
  return hand, discard_card

In [ ]:
def get_gemini_completion_for_prompt(model, prompt_input_text):
  full_gemini_prompt =f"""
    Você é um mestre de Uno e deve analisar um cenário de jogo e determinar a jogada correta ou o resultado, estritamente de acordo com as regras padrão do jogo Uno fornecidas abaixo. Mantenha a resposta com um MÁXIMO de 20-30 palavras, sendo o mais direto possível.

    {uno_rules}

    **Cenário de Jogo:**
    {prompt_input_text}

    **Responda de forma extremamente concisa, indicando apenas a jogada ou o resultado correto. Se houver uma breve justificativa de uma frase, inclua-a após a jogada/resultado. Não inclua qualquer frase introdutória como 'Jogada Correta:' ou 'Resultado:'**
    """
  response_gen = model.generate_content(full_gemini_prompt)

  return response_gen.text.strip()

In [ ]:
num_samples = 450
data_hands = []
data_discards = []
for _ in range(num_samples):
  hand, discard = generate_random_hand_and_discard(full_uno_deck)
  data_hands.append(hand)
  data_discards.append(discard)

In [ ]:
synthetic_dataset = []


In [ ]:
for i in range(num_samples):
  current_hand = data_hands[i]
  current_discard = data_discards[i]

  currente_hand_str = "[" + ",".join(current_hand)+"]"

  question = random.choice(["Qual carta você pode jogar?", "O que você pode fazer?", "O que acontece com você nesta rodada?",])

  prompt_input = f"Carta no descarte: {current_discard}, Cartas na mão: {currente_hand_str}. {question}"

  completion_from_gemini = get_gemini_completion_for_prompt(model_gemini_pro, prompt_input)
  synthetic_dataset.append({
        "prompt": prompt_input,
        "completion": completion_from_gemini
    })
  time.sleep(2)

In [ ]:
df = pd.DataFrame(synthetic_dataset)

In [ ]:
df

,prompt,completion
0,"Carta no descarte: Amarelo 7, Cartas na mão: [...",Jogue o Amarelo 7.
1,"Carta no descarte: Amarelo 7, Cartas na mão: [...",Amarelo 7. Combina com a carta descartada.
2,"Carta no descarte: Amarelo 7, Cartas na mão: [...",Amarelo 7. Combina com a cor e o número da ca...
3,"Carta no descarte: Azul 8, Cartas na mão: [Cor...",Vermelho 8. Combina com o número 8 no descarte.
4,"Carta no descarte: Verde +2, Cartas na mão: [A...",Jogue o Verde 1.
...,...,...
502,"Carta no descarte: Verde 2, Cartas na mão: [Ve...",Jogar o Vermelho 2. Combina com o número da c...
503,"Carta no descarte: Amarelo Inverter, Cartas na...",Comprar carta. Não há cartas na mão que combi...
504,"Carta no descarte: Verde Bloquear, Cartas na m...",Jogue o Amarelo +2.
505,"Carta no descarte: Vermelho 2, Cartas na mão: ...",Vermelho 4. Combina com o número 2 no descarte.


In [ ]:
# df.to_csv('/content/drive/MyDrive/Estudos/LLM_Fine-tuning/training_data.csv')

# Fine-Tuning

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Estudos/LLM_Fine-tuning/training_data.csv', index_col=0)

In [ ]:
df

,prompt,completion
0,"Carta no descarte: Amarelo 7, Cartas na mão: [...",Jogue o Amarelo 7.
1,"Carta no descarte: Amarelo 7, Cartas na mão: [...",Amarelo 7. Combina com a carta descartada.
2,"Carta no descarte: Amarelo 7, Cartas na mão: [...",Amarelo 7. Combina com a cor e o número da ca...
3,"Carta no descarte: Azul 8, Cartas na mão: [Cor...",Vermelho 8. Combina com o número 8 no descarte.
4,"Carta no descarte: Verde +2, Cartas na mão: [A...",Jogue o Verde 1.
...,...,...
502,"Carta no descarte: Verde 2, Cartas na mão: [Ve...",Jogar o Vermelho 2. Combina com o número da c...
503,"Carta no descarte: Amarelo Inverter, Cartas na...",Comprar carta. Não há cartas na mão que combi...
504,"Carta no descarte: Verde Bloquear, Cartas na m...",Jogue o Amarelo +2.
505,"Carta no descarte: Vermelho 2, Cartas na mão: ...",Vermelho 4. Combina com o número 2 no descarte.


In [ ]:
# Dataset Treatment

raw_dataset = Dataset.from_pandas(df)
train_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)['train']
eval_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)['test']

# Formatting the dataset to the SFTTrainer format
def formatting_prompts(examples):
    output_texts = []
    for i in range(len(examples['prompt'])):
        text = f"{examples['prompt'][i]}\n{examples['completion'][i]}{tokenizer_gemma.eos_token}"
        output_texts.append(text)
    return {"text": output_texts}

train_dataset_formatted = train_dataset.map(formatting_prompts, batched=True)
eval_dataset_formatted = eval_dataset.map(formatting_prompts, batched=True)


In [ ]:
train_dataset_formatted

Dataset({
    features: ['prompt', 'completion', '__index_level_0__', 'text'],
    num_rows: 456
})

In [ ]:
# LORA Configuration

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model_gemma = get_peft_model(model_gemma, lora_config)


In [ ]:
# Training configuration
from transformers import TrainingArguments

training_arguments = SFTConfig(
    output_dir="/content/drive/MyDrive/Estudos/LLM_Fine-tuning/models/gemma_uno_finetuned",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    seed=42,
    dataset_text_field='text',
    report_to="none",
    logging_steps=50,
)

trainer = SFTTrainer(
    model=model_gemma,
    train_dataset=train_dataset_formatted,
    eval_dataset=eval_dataset_formatted,
    args=training_arguments,
    peft_config=lora_config
)



In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.048900,0.480622
2,0.378800,0.466657
3,0.288200,0.453518
4,0.227900,0.473695
5,0.157200,0.608295
6,0.127300,0.579208
7,0.072900,0.691460
8,0.017100,0.805658
9,0.006200,0.889296
10,0.002200,0.917185


TrainOutput(global_step=570, training_loss=0.2081965192694936, metrics={'train_runtime': 1019.2364, 'train_samples_per_second': 4.474, 'train_steps_per_second': 0.559, 'total_flos': 1.408063731929088e+16, 'train_loss': 0.2081965192694936})

# Fine-tuned Model Evaluation

---



In [ ]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    '/content/drive/MyDrive/Estudos/LLM_Fine-tuning/models/gemma_uno_finetuned/checkpoint-171', # 3rd epoch, model without overfitting
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
trained_model = trained_model.merge_and_unload()

In [ ]:
# Save the merged model
trained_model.save_pretrained("/content/drive/MyDrive/Estudos/LLM_Fine-tuning/models/gemma_uno_finetuned/gemma_tuned_merged", safe_serialization=True)
tokenizer_gemma.save_pretrained("/content/drive/MyDrive/Estudos/LLM_Fine-tuning/models/gemma_uno_finetuned/gemma-7b-tuned-merged")

In [ ]:
trained_model = AutoModelForCausalLM.from_pretrained(
    '/content/drive/MyDrive/Estudos/LLM_Fine-tuning/models/gemma_uno_finetuned/gemma_tuned_merged',
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [39]:
trained_response_1 = generate_response(trained_model, prompt_1)
trained_response_2 = generate_response(trained_model, prompt_2)
trained_response_3 = generate_response(trained_model, prompt_3)
trained_response_4 = generate_response(trained_model, prompt_4)
trained_response_5 = generate_response(trained_model, prompt_5)

In [40]:
# Model examples
responses = [trained_response_1, trained_response_2, trained_response_3, trained_response_4, trained_response_5]

for prompt, response in zip(prompts, responses):
  print(f"**Prompt:**\n{prompt}\n")
  print(f"**Answer:**\n{response}\n")
  print("-" * 50)


**Prompt:**

Você está jogando Uno.
A carta no descarte é: Vermelho 7
Minha mão tem as seguintes cartas: [Azul 3, Vermelho 8, Verde 5, +2 Amarelo].
Qual carta você pode jogar?


**Answer:**
Vermelho 8.

--------------------------------------------------
**Prompt:**

Você está jogando Uno.
A carta no descarte é: Vermelho +2.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?


**Answer:**
Bloquear Vermelho.

--------------------------------------------------
**Prompt:**

Você está jogando Uno.
A carta no descarte é: Azul 4.
Sua mão tem as seguintes cartas: [Verde 0, Azul 1, Bloquear Vermelho].
Qual carta você pode jogar?


**Answer:**
Azul 1.

--------------------------------------------------
**Prompt:**

Você está jogando Uno.
Carta no descarte: Azul 8,
Cartas na mão: [Coringa,Vermelho 8,Amarelo +2,Verde 7,Vermelho 3,Vermelho 9,Amarelo 4].
Qual carta você pode jogar?


**Answer:**
Vermelho 8.

--------------------------------------------